In [36]:
from ProbRobNLP.logic import *
from collections import namedtuple, deque
from itertools import product
from sys import stderr
from pysat.solvers import Glucose3

In [2]:
# target: target predicate trying to be learned e.g. diagonal
# P_e set of extensional predicates i.e. fully defined by ground atoms. e.g. diagonal(x, y), in(x, e4), in(y,d5)
# arity_e: maps P_e and target to arities
# C set of constants. 

class LanguageFrame:
    def __init__(self, target, P_e, C):
        self.target = target
        self.P_e = P_e
        self.C = C
        self.arity = {p: p.arity for p in (P_e + [target])}

        


In [3]:
# L is a language frame
# B set of background assumptions formed from the predicates in P_e and C
# P positive examples
# N negative examples
class ILPProblem:
    def __init__(self, language, background, positive, negative):
        self.language = language
        self.background_assumptions = background
        self.positive = positive
        self.negative = negative
        

In [15]:
tau = namedtuple('tau', ["variables", "intensional"])

In [5]:
# PI describes a range of programs that can be generated
# P_a is a set of auxiliary intensional predicates. These are the additional invented predicates used to help define the target predicate
# arity_a maps P_a to N and is the arity of those predicates
# rules is a map from each intensional predicate to a par of rule templates (tau^1_p and tau^2_p)
# T specifies the max number of steps of foward chaining inference 
class ProgramTemplate:
    def __init__(self, P_a, rules, max_foward_chains):
        self.P_a = P_a
        self.rules = rules
        self.arity = [p.arity for p in P_a]
        self.max_forward_chains = max_foward_chains

In [18]:
def generate_clauses(target, t: tau, P_e, P_a):
    reset_variable()
    variables = [get_variable() for i in range(target.arity + t.variables)]
    
    head_variables = variables[:target.arity]
    
    target_atom = Atom(target, head_variables)
    
    P_a = P_a + [target]
    
    print(target_atom)
    
    possible_atoms = []
    
    for P in P_e:
        for terms in product(*[variables]*P.arity):
            
            possible_atoms.append(Atom(P, terms))
    if t.intensional:
        for P in P_a:
            for terms in product(*[variables]*P.arity):

                possible_atoms.append(Atom(P, terms))
    
    print(possible_atoms)
    
    clauses = []
    
    for body in product(possible_atoms, possible_atoms):
        
        body_variables = set()
        for p in body:
            body_variables.update(p.terms)
        
        clause = DefiniteClause(target_atom, set(body))
        
        unsafe = any([v not in body_variables for v in head_variables])
        circular = target_atom in body 
        #circular = any([target_atom == b for b in body])
        duplicated = clause in clauses
        has_intentional = int(len([b for b in body if b.pred in P_a]) > 0)
        satisfies_int = has_intentional == t.intensional
                
        
        if not unsafe and not circular and not duplicated and satisfies_int:
        
            clauses.append(clause)
    
    for clause in clauses:
        print(clause)
    print(len(clauses))
    
    return target_atom, possible_atoms

X, Y, Z = get_variable(), get_variable(), get_variable()
a = Atom(q, [X, Y])
b = Atom(q, [X, Y])
c = Atom(q, [X, Z])
a == b
a == c
a in (b, c)

q = Predicate('q', 2)
P_e = [Predicate('p', 2)]
constants = [Constant(c) for c in ['a', 'b', 'c', 'd']]
p = P_e[0]

L = LanguageFrame(p, P_e, constants)

a, b, c, d = constants
B = [Atom(p, [a,b]), Atom(p, [b,c]), Atom(p, [c,d])]
P = [Atom(q, [a,b]), Atom(q, [a,c]), Atom(q, [a,d]), Atom(q, [b,c]), Atom(q, [b,d]), Atom(q, [c,d])]
N = [Atom(q, [X, Y]) for X, Y in product(constants, constants) if Atom(q, [X, Y]) not in P]

tau_1 = tau(0, 0)
tau_2 = tau(1, 1)

generate_clauses(q, tau_1, P_e, [])
target, possible = generate_clauses(q, tau_2, P_e, [])

q(X0, X1)
[p(X0, X0), p(X0, X1), p(X1, X0), p(X1, X1)]
q(X0, X1) <- p(X0, X0), p(X0, X1)
q(X0, X1) <- p(X0, X0), p(X1, X0)
q(X0, X1) <- p(X0, X0), p(X1, X1)
q(X0, X1) <- p(X0, X1)
q(X0, X1) <- p(X0, X1), p(X1, X0)
q(X0, X1) <- p(X0, X1), p(X1, X1)
q(X0, X1) <- p(X1, X0)
q(X0, X1) <- p(X1, X0), p(X1, X1)
8
q(X0, X1)
[p(X0, X0), p(X0, X1), p(X0, X2), p(X1, X0), p(X1, X1), p(X1, X2), p(X2, X0), p(X2, X1), p(X2, X2), q(X0, X0), q(X0, X1), q(X0, X2), q(X1, X0), q(X1, X1), q(X1, X2), q(X2, X0), q(X2, X1), q(X2, X2)]
q(X0, X1) <- p(X0, X0), q(X1, X0)
q(X0, X1) <- p(X0, X0), q(X1, X1)
q(X0, X1) <- p(X0, X0), q(X1, X2)
q(X0, X1) <- p(X0, X0), q(X2, X1)
q(X0, X1) <- p(X0, X1), q(X0, X0)
q(X0, X1) <- p(X0, X1), q(X0, X2)
q(X0, X1) <- q(X1, X0), p(X0, X1)
q(X0, X1) <- p(X0, X1), q(X1, X1)
q(X0, X1) <- q(X1, X2), p(X0, X1)
q(X0, X1) <- p(X0, X1), q(X2, X0)
q(X0, X1) <- p(X0, X1), q(X2, X1)
q(X0, X1) <- p(X0, X1), q(X2, X2)
q(X0, X1) <- q(X1, X0), p(X0, X2)
q(X0, X1) <- p(X0, X2), q(X1, X1)
q(X0, X1

In [4]:
target
possible

for p in possible:
    print(target, p, target == p)

NameError: name 'target' is not defined

NameError: name 'q' is not defined

In [5]:
s = set() 
s.update([1])
s

{1}

In [6]:
q = Predicate('q', 2)
P_e = [Predicate('p', 2)]
constants = [Constant(c) for c in ['a', 'b', 'c', 'd']]
p = P_e[0]

In [7]:
L = LanguageFrame(p, P_e, constants)

In [8]:
a, b, c, d = constants

In [9]:
a, b, c, d = constants
B = [Atom(p, [a,b]), Atom(p, [b,c]), Atom(p, [c,d])]
P = [Atom(q, [a,b]), Atom(q, [a,c]), Atom(q, [a,d]), Atom(q, [b,c]), Atom(q, [b,d]), Atom(q, [c,d])]
N = [Atom(q, [X, Y]) for X, Y in product(constants, constants) if Atom(q, [X, Y]) not in P]

tau_1 = tau(0, 0)
tau_2 = tau(1, 1)

In [10]:
P = [Atom(q, [a,b]), Atom(q, [a,c]), Atom(q, [a,d]), Atom(q, [b,c]), Atom(q, [b,d]), Atom(q, [c,d])]

In [11]:
N = [Atom(q, [X, Y]) for X, Y in product(constants, constants) if Atom(q, [X, Y]) not in P]

In [12]:
tau_1 = tau(0, 0)
tau_2 = tau(1, 1)

NameError: name 'tau' is not defined

In [13]:
rules = {q: (tau_1, tau_2)}

NameError: name 'tau_1' is not defined

In [14]:
list(product([]))

[]

In [19]:
list(product(*[[1,2,3]]*3))

[(1, 1, 1),
 (1, 1, 2),
 (1, 1, 3),
 (1, 2, 1),
 (1, 2, 2),
 (1, 2, 3),
 (1, 3, 1),
 (1, 3, 2),
 (1, 3, 3),
 (2, 1, 1),
 (2, 1, 2),
 (2, 1, 3),
 (2, 2, 1),
 (2, 2, 2),
 (2, 2, 3),
 (2, 3, 1),
 (2, 3, 2),
 (2, 3, 3),
 (3, 1, 1),
 (3, 1, 2),
 (3, 1, 3),
 (3, 2, 1),
 (3, 2, 2),
 (3, 2, 3),
 (3, 3, 1),
 (3, 3, 2),
 (3, 3, 3)]

In [19]:
class SATInstance(object):
    def parse_and_add_clause(self, line):
        clause = []
        for literal in line.split():
            negated = 1 if literal.startswith('~') else 0
            variable = literal[negated:]
            if variable not in self.variable_table:
                self.variable_table[variable] = len(self.variables)
                self.variables.append(variable)
            encoded_literal = self.variable_table[variable] << 1 | negated
            clause.append(encoded_literal)
        self.clauses.append(tuple(set(clause)))

    def __init__(self):
        self.variables = []
        self.variable_table = dict()
        self.clauses = []

    @classmethod
    def from_file(cls, file):
        instance = cls()
        for line in file:
            line = line.strip()
            if len(line) > 0 and not line.startswith('#'):
                instance.parse_and_add_clause(line)
        return instance

    def literal_to_string(self, literal):
        s = '~' if literal & 1 else ''
        return s + self.variables[literal >> 1]

    def clause_to_string(self, clause):
        return ' '.join(self.literal_to_string(l) for l in clause)

    def assignment_to_string(self, assignment, brief=False, starting_with=''):
        literals = []
        for a, v in ((a, v) for a, v in zip(assignment, self.variables)
                     if v.startswith(starting_with)):
            if a == 0 and not brief:
                literals.append('~' + v)
            elif a:
                literals.append(v)
        return ' '.join(literals)

In [20]:
s = SATInstance()

In [21]:
s.parse_and_add_clause('A B ~C')
s.variables


['A', 'B', 'C']

In [22]:
s.variable_table

{'A': 0, 'B': 1, 'C': 2}

In [23]:
s.clauses

[(0, 2, 5)]

In [24]:
def setup_watchlist(instance):
    watchlist = [deque() for __ in range(2 * len(instance.variables))]
    for clause in instance.clauses:
        # Make the clause watch its first literal
        watchlist[clause[0]].append(clause)
    return watchlist

In [25]:
def update_watchlist(instance,
                     watchlist,
                     false_literal,
                     assignment,
                     verbose):
    """
    Updates the watch list after literal 'false_literal' was just assigned
    False, by making any clause watching false_literal watch something else.
    Returns False it is impossible to do so, meaning a clause is contradicted
    by the current assignment.
    """
    while watchlist[false_literal]:
        clause = watchlist[false_literal][0]
        found_alternative = False
        for alternative in clause:
            v = alternative >> 1
            a = alternative & 1
            if assignment[v] is None or assignment[v] == a ^ 1:
                found_alternative = True
                del watchlist[false_literal][0]
                watchlist[alternative].append(clause)
                break

        if not found_alternative:
            if verbose:
                dump_watchlist(instance, watchlist)
                print('Current assignment: {}'.format(
                      instance.assignment_to_string(assignment)),
                      file=stderr)
                print('Clause {} contradicted.'.format(
                      instance.clause_to_string(clause)),
                      file=stderr)
            return False
    return True

In [26]:
def solve(instance, watchlist, assignment, d, verbose):
    """
    Recursively solve SAT by assigning to variables d, d+1, ..., n-1. Assumes
    variables 0, ..., d-1 are assigned so far. A generator for all the
    satisfying assignments is returned.
    """
    if d == len(instance.variables):
        yield assignment
        return

    for a in [0, 1]:
        if verbose:
            print('Trying {} = {}'.format(instance.variables[d], a),
                  file=stderr)
        assignment[d] = a
        if update_watchlist(instance,
                            watchlist,
                            (d << 1) | a,
                            assignment,
                            verbose):
            for a in solve(instance, watchlist, assignment, d + 1, verbose):
                yield a

    assignment[d] = None

In [35]:
s = SATInstance()
s.parse_and_add_clause('A B ~C')
s.parse_and_add_clause('B C')
s.parse_and_add_clause('~B')
s.parse_and_add_clause('~A C')
watchlist = setup_watchlist(s)

solution = solve(s, watchlist, [0]*len(s.variables), 0, True)

In [34]:
print(list(solution))

[]


In [ ]:
g = Glucose3()
g.add_clause([-1, 2])
g.add_clause([-2, 3])
print(g.solve())
print(g.get_model())